# Procesamiento de datos con [*pandas*](https://pandas.pydata.org/)


Es una librería para **leer, manipular y procesar datos** con lenguaje Python 

Pandas provee
- Dos estructuras de datos *Dataframe* y *Series*
- Herramientas eficientes de análisis de datos que trabajan sobre estas estructuras

Pandas se combina bien con NumPy y Matplotlib 

Instala pandas si aun no lo haz hecho:

    conda install pandas
    
    pip install pandas --user
    
    sudo pacman -S python-pandas

In [ ]:
import pandas as pd
print("Versión de pandas "+pd.__version__)
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display

## Estructuras de datos de pandas

1. *Series* se usa para representar secuencias: Una columna de datos
1. *Dataframe* se usa para representar tablas: Múltiples columnas de datos


- Las filas y las columnas están etiquetadas con **índices**
- Por defecto el índice es número entero que parte de cero pero podemos especificarlo como queramos
- Soportan múltiples tipos y NaNs (missing data)

Por ejemplo una [serie](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html):

In [ ]:
s = pd.Series(["asd", 4521, 24.2142])
print(s)
[type(x) for x in s]

In [ ]:
s = pd.Series([3.124, 5.124, 2.1416, 10.])
print(s)
[type(x) for x in s]

In [ ]:
# Atributos index y values
display(s.index, 
        s.values, 
        s.describe())

In [ ]:
display(s[:2], # slicing
        s[-2:], 
        s[[0, 2, 3]], # fancy-indexing
        s[s>5]) # mascaras

En los ejemplos anteriores el índice se creo de forma automática pero podemos especificarlo con

In [ ]:
s = pd.Series([3.124, 5.124, 2.1416, 10.], index=[1, 0, 2, 40])
display(s, s[40])

**IMPORTANTE** El índice no tiene que ser númerico

Idealmente el índice debería ser informativo por si mismo, por ejemplo:

In [ ]:
s_IPSA = pd.Series([0.36, -0.31, -0.6, 0.0], 
                   index=["AGUAS-A", "BSANTANDER", "CMPC", "ENTEL"],
                   name='% variación')
display(s_IPSA)

In [ ]:
# A diferencia de un diccionario de Python, la serie soporta slicing (tiene orden)
display(s_IPSA["BSANTANDER":"CMPC"])
# E indexación con otro arreglo
display(s_IPSA[["CMPC", "AGUAS-A"]])

In [ ]:
# Se puede construir una serie a partir de escalares, listas, diccioanrios y ndarray

display(pd.Series(2.14159, index=np.arange(5)), 
        pd.Series([4, 3, 2, 1, 0]),
        pd.Series(np.random.randn(5)),
        pd.Series({'Valdivia': 143207, 'Santiago': 5614000}))

¿Qué podemos hacer con la [serie](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)?

In [ ]:
display(s_IPSA)
print("Aritmética:")
display(s_IPSA+2, s_IPSA*2, s_IPSA**2)

In [ ]:
print("Reducciones y estadísticos:")
display(s_IPSA.sum(), s_IPSA.cumsum().values, 
        s_IPSA.mean(), s_IPSA.std(), 
        s_IPSA.min(), s_IPSA.idxmin())

In [ ]:
print("Operaciones lógicas:")
display(s_IPSA<0, # Equivalente a s.lt(0)
        (s_IPSA>-0.5) & (s_IPSA<0.2))

También podemos graficarlo

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), tight_layout=True)
ax.set_ylabel(s_IPSA.name);

s_IPSA.plot(ax=ax, kind='bar')
#ax.bar(x=s.index, height=s.values)

# kind = {line, bar, barh, hist, box, kde, area, pie}

## Dataframes

- Un *dataframe* es una tabla
- Es un conjunto de *series* que comparten índice de fila
- Cada serie es una columna del *dataframe*
- El ambiente jupyter imprime un dataframe como una tabla markdown de forma automática


### Creando un dataframe

A partir de una serie

In [ ]:
df = pd.DataFrame(s_IPSA)
df

In [ ]:
# Atributos informativos
print(df.dtypes)
print("")
print(df.info())

A partir de múltiples series

In [ ]:
s2 = pd.Series([1653, 3531, 5998, 1408], 
              index=s_IPSA.index,
              name='Monto M$')
s3 = pd.Series(["CL0000000035", "CLP1506A1070", "CL0000001314", "CLP37115105"], 
              index=s_IPSA.index,
              name='ISIN')

df_IPSA = pd.concat([s_IPSA, s2, s3], axis=1)
df_IPSA

A partir de un ndarray bidimensional

In [ ]:
pd.DataFrame(np.eye(3), columns=["a", "b", "c"])

O de una lista de diccionarios

In [ ]:
d = [{'nombre': nombre, 'apellido': apellido} for nombre, apellido in zip(["pablo", "fulano"], ["huijse", "de tal"])]
pd.DataFrame(d)

### Indices

- Un dataframe tiene dos sets de índices: fila y columna
- Ambos puede ser enteros o etiquetas

Podemos recuperarlos con

In [ ]:
mis_columnas = df_IPSA.columns
mis_filas = df_IPSA.index
display(mis_columnas, mis_filas)

Podemos manipularlos como ndarrays

In [ ]:
display(mis_columnas[0], 
        mis_columnas[::-1], 
        mis_filas[::2], 
        mis_filas.shape)

Los índices son tipos inmutables

In [ ]:
# mis_columnas[0] = "asd"

Los [índices](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.html) soportan operaciones lógicas (tipo Set)

In [ ]:
# Nota: idx1 es equivalente a 
# pd.Index(np.arange(10)) y pd.Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) pero ahorra memoria
idx1 = pd.Index(range(10))
idx2 = pd.Index(range(0, 20, 2))
# display(idx1, idx2)
display(idx1.intersection(idx2),  # Equivalente a: idx1 & idx2
        idx1.union(idx2), # TEquivalente a: idx1 | idx2
        idx1.symmetric_difference(idx2), # Equivalente a: idx1 ^ idx2
        idx1.difference(idx2),
        idx2.difference(idx1)) 

### Slicing e indexación  de dataframes

Podemos obtener una serie a partir de una columna

In [ ]:
df_IPSA["ISIN"]
# También se puede llamar como atributo df.ISIN pero esto es menos recomendable

Podemos obtener una sub-tabla seleccionando columnas

In [ ]:
display(df_IPSA[["ISIN"]], 
        df_IPSA[["Monto M$", "% variación"]])

Podemos extraer una o más filas usando los atributos `loc[]` o `iloc[]`

`loc[]` acepta etiquetas, `iloc[]` acepta el índice "interno" (entero)

Cuando extraemos sólo una fila se obtiene una serie, de lo contrario se obtiene un dataframe

In [ ]:
display(df_IPSA.loc["CMPC"],
        df_IPSA.loc[["CMPC", "AGUAS-A"]],
        df_IPSA.loc["AGUAS-A":"CMPC"])

In [ ]:
display(df_IPSA.iloc[2], 
        df_IPSA.iloc[[2, 0]],
        df_IPSA.iloc[:3])

No confundirse con `iloc[]` y `loc[]`

In [ ]:
np.random.seed(0)
stupid_dataframe = pd.DataFrame(np.random.permutation(16).reshape(4, 4), 
                  index=[2, 0, 3, 1])

display(stupid_dataframe, 
        stupid_dataframe.loc[2],
        stupid_dataframe.iloc[2])

También podemos recuperar simultaneamente un sub-conjunto de filas y columnas

    df.loc[filas, columnas]
    
Se pueden hacer slicing, fancy-indexing, máscaras, etc

In [ ]:
display(df_IPSA.loc["CMPC", "Monto M$"],
        df_IPSA.iloc[2, 1],
        df_IPSA.loc[:"CMPC", "Monto M$":], 
        df_IPSA.iloc[:3, 1:],
        df_IPSA.loc[["ENTEL", "CMPC"], ["Monto M$", "% variación"]], 
        df_IPSA.loc[df_IPSA["Monto M$"] > 2000, "Monto M$":])

Notemos la diferencia de tiempo entre

In [ ]:
%timeit -n10 df_IPSA.loc["AGUAS-A":"CMPC"][["Monto M$", "ISIN"]]
%timeit -n10 df_IPSA[["Monto M$", "ISIN"]].loc["AGUAS-A":"CMPC"]
%timeit -n10 df_IPSA.loc["AGUAS-A":"CMPC", "Monto M$":] 
%timeit -n10 df_IPSA.iloc[:2, 1:] 

Si quieres una columna completa no uses `loc`/`iloc`

In [ ]:
%timeit -n10 df_IPSA["ISIN"]
%timeit -n10  df_IPSA.loc[:, "ISIN"]
%timeit -n10  df_IPSA.iloc[:, 2]

#### Evitar posible confusión

In [ ]:
# Esto se puede hacer
display(df_IPSA["Monto M$"])
# pero esto no: 
#display(df_IPSA["Monto M$":])

# Esto se puede hacer
display(df_IPSA["BSANTANDER":])
# y es equivalente a df_IPSA.loc["BSANTANDER":]
# Pero esto no: 
#df_IPSA["BSANTANDER"]

## Operaciones sobre [dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)

Convertir a ndarray:

In [ ]:
df_IPSA.to_numpy() # Equivalente a df_IPSA.values

Intercambiar filas y columnas (trasponer):

In [ ]:
df_IPSA.T

**Aritmética sobre dataframes**

Podemos hacer operaciones aritméticas simples y/o aplicar funciones de NumPy

Notemos que no se pueden operar strings de forma directa:
- Podemos usar [`select_dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html) para obtener un subconjunto de datos de tipo númerico
- Podemos usar *parsear* el atributo string a numérico (más adelante detallaremos esto)

In [ ]:
df_numeric = df_IPSA.select_dtypes(np.number)
display(df_numeric,
        df_numeric + 2, # Equivalente a df_numeric.add(2)
        df_numeric*2, # Equivalente a df_numeric.multiply(2)
        np.log(np.abs(df_numeric)+1e-10))

**Alineamiento:** Las operaciones entre columnas se guian por el índice

- Si un índice no es compartido por ambos operandos se retorna un NaN
- Podemos evitar que esto ocurre seteando el `fill_value`

In [ ]:
df1 = df_IPSA.loc["BSANTADER":, "Monto M$"]
df2 = df_IPSA.loc[:"CMPC", "% variación"]
display(df1,
        df2,
        df1/df2, # Equivalente a df1.divide(df2)
        df1.divide(df2, fill_value=1))

**Manejando *missing values* (NaNs)**

Podemos eliminar o rellenar los NaNs antes de operar

In [ ]:
df = pd.DataFrame([[1, 2, 3, None, 5], 
                   [np.nan, 1.154, 5.12, 403, "asd"], 
                   [x for x in range(5)]])
display(df,
        df.dropna(),  # Elimina filas y columnas con NaN
        df.dropna(axis='columns', how='any'),
        df.fillna(0), # Rellena NaN
        df.notnull()) # Mascara booleana, también existe isnull()

**Reducciones**

Las atributos de reducción que vimos para *Series* también se pueden usar con *DataFrames*

In [ ]:
display(df_IPSA.count(),
        df_IPSA.sum(),
        df_numeric.sum(),
        df_numeric.prod(),
        df_numeric.std(), 
        df_numeric.max(),  
        df_numeric.idxmax())  

### Gráficos

Se pueden usar gráficos sencillos directamente de un [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4), tight_layout=True)
df_IPSA.plot(x=None, y=["Monto M$", "% variación"], 
             ax=ax, kind='line', subplots=True);